# Set Up NVFLARE in POC Mode

[POC mode](https://nvflare.readthedocs.io/en/main/user_guide/poc_command.html) allows users to test the features of a full FLARE deployment on a single machine, without the overhead of a true distributed deployment and the need to establish secure communication between server and client systems.

Compared to the FL Simulator, where the job run is automated on a single system, POC mode allows you to establish and connect distinct server and client "systems" which can then be orchestrated using the FLARE Console.  This can be useful in preparation for a distributed deployment.

>**Warning**
>POC mode is not intended to be secure and should not be run in any type of production environment or any environment where the server’s ports are exposed. For actual deployment and even development, it is recommended to use a secure provisioned setup or the FL Simulator.

To get started, let's look at the NVFlare CLI usage for the poc subcommand:

In [ ]:
!nvflare poc -h

### Preparing the POC environment
Before running POC mode, there are a couple important environment variables that should be set.

First, to simplify deploying the example apps in the NVFlare GitHub repo, you can set `NVFLARE_HOME` to the root of the GitHub clone.  In this case, we've cloned to our current working directory, so we can set it as:

In [ ]:
import os
workdir=os.getcwd()
%env NVFLARE_HOME={workdir}/../NVFlare

By default, POC mode uses a temporary workspace in /tmp/nvflare/poc.

If you prefer to set a workspace in different location than the default, you can set the environment variable:

```
export NVFLARE_POC_WORKSPACE = /you/new/location/path

```

For this notebook, we would like to keep the workspace within our working directory, so let's create a poc_workspace dir.  We can then use the `NVFLARE_POC_WORKSPACE` environment variable to define this as the POC workspace.

Note:  if you have previously created the poc_workspace, you will want to clean it up using the `nvflare poc --clean` command or manually remove the `poc_workspace` directory.

In [ ]:
# !nvflare poc --clean
!rm -r poc_workspace
!mkdir poc_workspace
%env NVFLARE_POC_WORKSPACE={workdir}/poc_workspace

### Preparing the POC workspace

Now that we've configured out POC environment, we can prepare the POC workspace.  By default, the following command will generate POC packages for a server and two clients.

```
nvflare poc --prepare 
```

You can specify a different number of clients with the -n option:

``` 
   nvflare poc --prepare -n N
```

If you running the POC prepare command, there is a prompt for user to answer. We are going to pre-answer as Yes in the following:

In [ ]:
! echo 'y' |  nvflare poc --prepare

In [ ]:
!tree poc_workspace

### Start Flare system in POC

When starting the POC deployment, it's necessary to use a separate terminal since the `nvflare poc --start` command will run  in the foreground emitting output from the server and any connected clients.

Also note that `nvflare poc --start` starts all participants, including the admin console. We want to start the 2 clients and 1 server, but without the FLARE console (aka Admin Console), as we are going to interact with the system from a notebook.

Create a terminal to execute the following command (in JupyterLab or host terminal), then

```
  nvflare poc --start -ex admin
```

In [ ]:
%%bash --bg  

nvflare poc --start -ex admin
echo "sleep for few seconds to let system start"
sleep 3

In [ ]:
! ps -eaf | grep nvflare

### Check System information

In [ ]:

import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(workspace, "admin")
sess = new_insecure_session(admin_dir)
sys_info = sess.get_system_info()

print(f"Server info:\n{sys_info.server_info}")
print("\nclient_info")
for client in sys_info.client_info:
    print(client)

Once the system is up, we are now ready to go back to the notebook to work on your project.

### Stop FLARE system in POC

Once you are done with the FLARE system, you can shutdown the poc system.
From terminal, issue the following command stop poc in the terminal
```
   nvflare poc --stop
```


### Cleanup POC workspace

In [ ]:
! nvflare poc --clean